# Methods


In [1]:
function f(x::Any)
    gamma(x+1)
end

f (generic function with 1 method)

In [2]:
function f(n::Integer)
    factorial(n)
end

f (generic function with 2 methods)

In [3]:
f(3.0)

6.0

In [4]:
f(3)

6

In [5]:
f(im)

0.49801566811835474 - 0.15494982830181012im

In [6]:
f(-2)

LoadError: DomainError:

In [7]:
factorial(sqrt(2))

1.2538154806428916

In [8]:
f(sqrt(2))

1.2538154806428916

# Interval Arithmetic

In [2]:
struct Interval{T<:Real} <: Number
  lo::T
  hi::T
end

(a::Real)..(b::Real) = Interval(a,b)

Base.show(io::IO, iv::Interval) = print(io, "($(iv.lo))..($(iv.hi))")

In [3]:
1..2

(1)..(2)

In [4]:
typeof(ans)

Interval{Int64}

In [5]:
sizeof(1..2) 

16

In [6]:
(1.5)..(2.5)

(1.5)..(2.5)

In [7]:
typeof(ans)

Interval{Float64}

In [8]:
(1//2)..(2//3)

(1//2)..(2//3)

In [9]:
typeof(ans)

Interval{Rational{Int64}}

In [10]:
sizeof((1//2)..(2//3)) 

32

In [11]:
methods(Interval)

# 5 methods for generic function "(::Type)":
[1] (::Type{Interval})(lo::T, hi::T) where T<:Real in Main at In[2]:2
[2] (::Type{T})(x::T) where T<:Number in Core at boot.jl:725
[3] (::Type{T})(x::Base.TwicePrecision) where T<:Number in Base at twiceprecision.jl:243
[4] (::Type{T})(x::AbstractChar) where T<:Union{AbstractChar, Number} in Base at char.jl:50
[5] (::Type{T})(arg) where T in Base at deprecated.jl:463

In [13]:
try
    (0.5)..(1)
catch e
    println(e)
end

MethodError(Interval, (0.5, 1), 0x0000000000006c52)


In [14]:
Interval(lo::Real, hi::Real) = Interval(promote(lo,hi)...)

Interval

In [15]:
(0.5)..1

(0.5)..(1.0)

In [16]:
1..π

(1.0)..(3.141592653589793)

In [18]:
ℯ..π

(2.718281828459045)..(3.141592653589793)

In [31]:
import Base: +, -
+(a::Interval, b::Interval) = (a.lo + b.lo)..(a.hi + b.hi)
-(a::Interval, b::Interval) = (a.lo - b.hi)..(a.hi - b.lo)

- (generic function with 192 methods)

In [32]:
(2..3) + (-1..1)

(1)..(4)

In [33]:
(2..3) - (1..2)

(0)..(2)

In [35]:
import Base: convert, promote_rule #<== allows extending them

convert(::Type{Interval{T}}, x::Real) where {T<:Real} = (x = convert(T,x); x..x)
convert(::Type{Interval{T}}, iv::Interval) where {T<:Real} =
    convert(T,iv.lo)..convert(T,iv.hi)

promote_rule(::Type{Interval{A}}, ::Type{B}) where {A<:Real,B<:Real} = 
    Interval{promote_type(A,B)}
promote_rule(::Type{Interval{A}}, ::Type{Interval{B}}) where {A<:Real,B<:Real} =
    Interval{promote_type(A,B)}

promote_rule (generic function with 135 methods)

In [36]:
1..2 + 1

(1)..(3)

In [37]:
1..2 + 1.5

(1.0)..(3.5)

In [38]:
3 - 1..2 + 2//3

(2//1)..(8//3)

In [39]:
big(2)^100 + (1//3)..(2//3)

(3802951800684688204490109616129//3)..(2//3)

In [40]:
typeof(ans)

Interval{Rational{BigInt}}

In [41]:
@which big(2)^100 + (1//3)..(2//3)

..(a::Real, b::Real) in Main at In[2]:6

In [46]:
±(a::Real    , b::Real )    = (a - b)..(a + b)
±(a::Interval, b::Interval) = (a.lo - b.hi)..(a.hi + b.hi)
±(a::Number  , b::Number )  = ±(promote(a,b)...)

± (generic function with 3 methods)

In [47]:
1 ± 2

(-1)..(3)

In [48]:
π - (2 ± 1//2)

LoadError: MethodError: no method matching real(::Interval{Rational{Int64}})
Closest candidates are:
  real(!Matched::Missing) at missing.jl:79
  real(!Matched::Complex) at complex.jl:63
  real(!Matched::Real) at complex.jl:77
  ...